_Lambda School Data Science_

# Make features

Objectives
-  understand the purpose of feature engineering
-  work with strings in pandas
- work with dates and times in pandas

Links
- [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)
- Python Data Science Handbook
  - [Chapter 3.10](https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html), Vectorized String Operations
  - [Chapter 3.11](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html), Working with Time Series

## Get LendingClub data

[Source](https://www.lendingclub.com/info/download-data.action)

In [1]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2019-03-28 19:01:09--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [               <=>  ]  21.29M   806KB/s    in 28s     

2019-03-28 19:01:37 (793 KB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [22329081]



In [2]:
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [3]:
# These are linux commands
!head LoanStats_2018Q4.csv

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","op

In [4]:
!tail LoanStats_2018Q4.csv

"","","5600","5600","5600"," 36 months"," 13.56%","190.21","C","C1","","n/a","RENT","15600","Not Verified","Oct-2018","Current","n","","","credit_card","Credit card refinancing","836xx","ID","15.31","0","Aug-2012","0","","97","9","1","5996","34.5%","11","w","5083.61","5083.61","750.29","750.29","516.39","233.90","0.0","0.0","0.0","Feb-2019","190.21","Mar-2019","Feb-2019","0","","1","Individual","","","","0","0","5996","0","0","0","1","20","0","","0","2","3017","35","17400","1","0","0","3","750","4689","45.5","0","0","20","73","13","13","0","13","","20","","0","3","5","4","4","1","9","10","5","9","0","0","0","0","100","25","1","0","17400","5996","8600","0","","","","","","","","","","","","N","","","","","","","","","","","","","","","Cash","N","","","","","",""
"","","23000","23000","23000"," 36 months"," 15.02%","797.53","C","C3","Tax Consultant","10+ years","MORTGAGE","75000","Source Verified","Oct-2018","Charged Off","n","","","debt_consolidation","Debt consolidation","352xx","AL","

## Load LendingClub data

pandas documentation
- [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)
- [`options.display`](https://pandas.pydata.org/pandas-docs/stable/options.html#available-options)

In [0]:
# Feautures are essentially another fancy term for specified columns

In [0]:
import pandas as pd
df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1, skipfooter=2, engine='python')

In [6]:
df.info() # That weird column is throwing things off

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128412 entries, 0 to 128411
Columns: 145 entries, id to settlement_term
dtypes: float64(57), int64(51), object(37)
memory usage: 142.1+ MB


In [0]:
df.columns.tolist() # see all columns

In [7]:
df.isnull().sum().sum()

5229357

In [0]:
# How to extend rows
pd.options.display.max_columns = 500
pd.options.display.max_columns = 500
df.head()

## Work with strings

For machine learning, we usually want to replace strings with numbers.

We can get info about which columns have a datatype of "object" (strings)

In [9]:
df.describe(exclude='number')

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,verification_status_joint,sec_app_earliest_cr_line,hardship_flag,hardship_type,hardship_reason,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
count,128412,128412,128412,128412,107465,116708,128412,128412,128412,128412,128412,128412,128412,128412,128412,128412,128256,128412,128250,124941,128411,128412,14848,16782,128412,1,1,1,1,1,1,1,128412,128412,2,2,2
unique,2,46,7,35,43892,11,4,3,3,6,2,12,12,880,50,644,1074,2,5,3,7,2,3,573,2,1,1,1,1,1,1,1,2,2,1,1,1
top,36 months,13.56%,A,A4,Teacher,10+ years,MORTGAGE,Not Verified,Oct-2018,Current,n,debt_consolidation,Debt consolidation,112xx,CA,Aug-2006,0%,w,Feb-2019,Mar-2019,Feb-2019,Individual,Not Verified,Aug-2006,N,INTEREST ONLY-3 MONTHS DEFERRAL,UNEMPLOYMENT,ACTIVE,Feb-2019,Apr-2019,Feb-2019,Late (16-30 days),Cash,N,Feb-2019,ACTIVE,Feb-2019
freq,88179,6976,38011,9770,2090,38826,63490,58350,46305,123768,128411,70603,70603,1370,17879,1130,1132,114498,123797,124903,125061,111630,6360,155,128411,1,1,1,1,1,1,1,102516,128410,2,2,2


### Convert `int_rate`



In [10]:
'13.56%'.strip('%') # How you strip symbols from a str

'13.56'

In [11]:
type('13.56%'.strip('%')) # still a string

str

In [12]:
float('13.56%'.strip('%')) # put all of this within a float and that accomplishes the job

13.56

In [0]:
# This is the one-liner that you really want to work with, most optimal

df['int_rate'] = df['int_rate'].str.strip('%').astype(float) 
#df['int_rate'].str.strip('%').astype(float).head() # Now it is a float, although not perm


In [14]:
 df['int_rate'].head() # Now it is a perm. float

0    13.56
1    18.94
2    17.97
3    18.94
4    16.14
Name: int_rate, dtype: float64

In [0]:
#(df['int_rate'] / 100).head # Vectorizing the DF

Define a function to remove percent signs from strings and convert to floats

In [55]:
string = '13.56%'

def remove_percent(string):
  return float(string.strip('%'))

remove_percent(string)

13.56

Apply the function to the `int_rate` column

In [0]:
# Both of those methods return the same results
# You got an error, because you already did it above

#df['int_rate'] = df['int_rate'].apply(remove_percent).head()


### Clean `emp_title`

Look at top 20 titles

In [56]:
# Looking at the Top 20, just in the DS
df['emp_title'].head(20)

# Sometimes, you may want to combine titles 
df['emp_title'].value_counts().head(20) # These are the top 20 positions, organized

Teacher                     2090
Manager                     1773
Registered Nurse             952
Driver                       924
RN                           726
Supervisor                   697
Sales                        580
Project Manager              526
General Manager              523
Office Manager               521
Owner                        420
Director                     402
Operations Manager           387
Truck Driver                 387
Nurse                        326
Engineer                     325
Sales Manager                304
manager                      301
Supervisor                   270
Administrative Assistant     269
Name: emp_title, dtype: int64

How often is `emp_title` null?

In [57]:
df['emp_title'].isnull().sum()

20947

Clean the title and handle missing values

In [58]:
import numpy as np
# I want to have consistant Capitalization
# Clean trailing values -- strip spaces
# Replace NaN with 'Missing'
# This can be used for various things, numbers etc

examples = ['owner', 'Supervisor ',
            ' Project Manager', np.nan]

def clean_title(x):
  if isinstance(x, str):
   
    return x.strip().title()
  
  else: 
    return 'Robot'
  
[clean_title(x) for x in examples]

['Owner', 'Supervisor', 'Project Manager', 'Robot']

In [0]:
# This is applying our previous function to all of the data in Emp Title
# Overwrite it & Assign back to that col
df['emp_title'] = df['emp_title'].apply(clean_title)

In [0]:
df['emp_title'].value_counts().head(20)

### Create `emp_title_manager`

pandas documentation: [`str.contains`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html)

In [0]:
# Specify where Manager is
# Then add a new column to show where that is a true/ false col
df['emp_title_manager'] = df['emp_title'].str.contains('Manager')

In [0]:
# verify with shape

In [62]:
# Normalize shows percentage
df['emp_title_manager'].value_counts(normalize=True)

False    0.860745
True     0.139255
Name: emp_title_manager, dtype: float64

In [63]:
df.groupby('emp_title_manager')['int_rate'].mean()

emp_title_manager
False    12.958054
True     12.761840
Name: int_rate, dtype: float64

In [64]:
df['emp_title'].value_counts().head()

Robot               20947
Teacher              2557
Manager              2395
Registered Nurse     1418
Driver               1258
Name: emp_title, dtype: int64

In [0]:
# This shows how to represent that by a percentage
df.isnull().sum().sort_values(ascending=False) / len(df)


## Work with dates

pandas documentation
- [to_datetime](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)
- [Time/Date Components](https://pandas.pydata.org/pandas-docs/stable/timeseries.html#time-date-components) "You can access these properties via the `.dt` accessor"

In [66]:
df['issue_d'].head().values

array(['Dec-2018', 'Dec-2018', 'Dec-2018', 'Dec-2018', 'Dec-2018'],
      dtype=object)

In [67]:
df['issue_d'].describe()

count       128412
unique           3
top       Oct-2018
freq         46305
Name: issue_d, dtype: object

In [0]:
df['issue_d'] = pd.to_datetime(df['issue_d'], infer_datetime_format=True)

In [69]:
df['issue_d'].head().values

array(['2018-12-01T00:00:00.000000000', '2018-12-01T00:00:00.000000000',
       '2018-12-01T00:00:00.000000000', '2018-12-01T00:00:00.000000000',
       '2018-12-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [70]:
df['issue_d'].describe()

count                  128412
unique                      3
top       2018-10-01 00:00:00
freq                    46305
first     2018-10-01 00:00:00
last      2018-12-01 00:00:00
Name: issue_d, dtype: object

In [0]:
# Specify year and month and create new cols
df['issue_year'] = df['issue_d'].dt.year
df['issue_month'] = df['issue_d'].dt.month

In [72]:
# Random Values
df['issue_month'].sample(n=10).values

array([12, 12, 12, 12, 11, 10, 12, 12, 11, 12])

In [73]:
# Time elapsed
#df['earliest_cr_line'].head()

df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'],
                                        infer_datetime_format=True)

# See how many days from ECL 
df['earliest_cr_line'].head()

0   2001-04-01
1   1987-06-01
2   2011-04-01
3   2006-02-01
4   2000-12-01
Name: earliest_cr_line, dtype: datetime64[ns]

In [0]:
# Difference between the 2 dates
# Really cool
df['days_from_earliest_credit_to_issue'] = (df['issue_d'] - df['earliest_cr_line']).dt.days


In [75]:
df['days_from_earliest_credit_to_issue'].describe()

count    128412.000000
mean       5859.891490
std        2886.535578
min        1126.000000
25%        4049.000000
50%        5266.000000
75%        7244.000000
max       25171.000000
Name: days_from_earliest_credit_to_issue, dtype: float64

In [76]:
25171 / 365

68.96164383561644

In [77]:
# Creating your own code to look for certain cols
[col for col in df if col.endswith('_d')]

['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

In [0]:
for col in ['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']:
  df[col] = pd.to_datetime(df[col], infer_datetime_format=True)

In [79]:
df.describe(include='datetime')

,issue_d,earliest_cr_line,last_pymnt_d,next_pymnt_d,last_credit_pull_d
count,128412,128412,128250,124941,128411
unique,3,644,5,3,7
top,2018-10-01 00:00:00,2006-08-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-02-01 00:00:00
freq,46305,1130,123797,124903,125061
first,2018-10-01 00:00:00,1950-01-01 00:00:00,2018-10-01 00:00:00,2019-02-01 00:00:00,2018-08-01 00:00:00
last,2018-12-01 00:00:00,2015-11-01 00:00:00,2019-02-01 00:00:00,2019-04-01 00:00:00,2019-02-01 00:00:00


# ASSIGNMENT

- Replicate the lesson code.

- Convert the `term` column from string to integer. (*DONE)

- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.

- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

### **Convert the 'term' column from string to integer**

In [0]:
# I did not use the .astype fucntion because the dtype was already an int, just needed months removed
df['term'] = df['term'].str.strip('months')

In [26]:
# It works!
df['term'].value_counts()

 36     3
 60     2
Name: term, dtype: int64

### **Make a column named 'loan_status_is_great'. It should contain:

*   The integer 1 if 'loan_status' is 'Current' or 'Fully Paid'
*   Else, it should contain 0

In [80]:
df['loan_status'].value_counts(dropna=False)

Current               123768
Fully Paid              3438
Late (31-120 days)       509
In Grace Period          441
Late (16-30 days)        223
Charged Off               33
Name: loan_status, dtype: int64

In [0]:
# Kept getting an error when I tried to take this out of one-line 
# I am sure I could have simplified this by creating a dictionary or if/else 
# But this worked, so moving on

df['loan_status_is_great'] = df['loan_status'].replace('Current', 1).replace('Fully Paid', 1).replace('Late (31-120 days)', 0).replace('In Grace Period', 0).replace('Late (16-30 days)', 0 ).replace('Charged Off', 0)


In [82]:
# It worked!
df['loan_status_is_great'].value_counts()

1    127206
0      1206
Name: loan_status_is_great, dtype: int64

### **Make 'last_pymnt_d_month' and 'last_pymnt_d_year' columns**

In [0]:
# Years Only
df['last_pymnt_d_year'] = df['last_pymnt_d'].dt.year

# Months Only
df['last_pymnt_d_month'] = df['last_pymnt_d'].dt.month

In [94]:
df['last_pymnt_d_year'].head()

0    2019.0
1    2019.0
2    2019.0
3    2019.0
4    2019.0
Name: last_pymnt_d_year, dtype: float64

In [92]:
df['last_pymnt_d_month'].head()

0    2.0
1    2.0
2    2.0
3    2.0
4    2.0
Name: last_pymnt_d_month, dtype: float64

# STRETCH OPTIONS

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01